Através dessa análise é possível obter diversos insights ...... Data set laranjas...

## Metodologia

### 1. Obtenção de Dados

#### 1.1 Importação de bibliotecas

In [34]:
import pandas as pd
from IPython.display import display, Markdown, HTML
import joblib
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 2.0.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 18,

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

#### 1.2 Importação dos Dados

In [19]:
df = pd.read_csv("../data/raw/Orange_Quality_Data.csv")
dict = pd.read_csv("../data/external/dicionario.csv")

display(df.describe(include='all'))
display(Markdown(
    "***adicionar infos básicas aqui***\n\n"
    "---"
))
display(dict)
display(Markdown(
    "Temos 11 variáveis e 241 amostras.\n" 
    "* Temos 5 variáveis Quantitativas, divididas em 4 Contínuas e 1 Discreta.\n"
    "* Temos 6 variáveis Qualitativas, divididas igualmente em Nominal e Ordinal.\n" 
))

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
count,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000,241,241,241,241.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,24,12,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deep Orange,Cara Cara,N,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,21,149,NaN
mean,7.844813,205.128631,10.907884,3.473900,3.072614,15.344398,3.599585,NaN,NaN,NaN,3.817427
std,1.086002,56.461012,2.760446,0.421007,1.323630,5.323852,1.205214,NaN,NaN,NaN,1.014410
min,6.000000,100.000000,5.500000,2.800000,1.000000,4.000000,1.000000,NaN,NaN,NaN,1.000000
25%,6.900000,155.000000,8.500000,3.200000,2.000000,11.000000,3.000000,NaN,NaN,NaN,3.000000
50%,7.800000,205.000000,11.000000,3.400000,3.000000,15.000000,4.000000,NaN,NaN,NaN,4.000000
75%,8.700000,252.000000,13.400000,3.800000,4.000000,20.000000,4.500000,NaN,NaN,NaN,4.500000


***adicionar infos básicas aqui***

---

,variavel,descrição,tipo,subtipo
0,Size,Tamanho da fruta em cm,Quantitativa,Contínua
1,Weight,Peso da fruta em g,Quantitativa,Contínua
2,Brix,Nível de doçura,Quantitativa,Contínua
3,pH,Nível de acidez em pH,Quantitativa,Contínua
4,Softness,Maciez de 1-5,Qualitativa,Ordinal
5,HarvestTime,Dias desde a colheita,Quantitativa,discreta
6,Ripeness,Maduração de 1-5,Qualitativa,Ordinal
7,Color,Cor da laranja,Qualitativa,Nominal
8,Variety,Variedade da laranja,Qualitativa,Nominal
9,Blemishes,Defeito,Qualitativa,Nominal


Temos 11 variáveis e 241 amostras.
* Temos 5 variáveis Quantitativas, divididas em 4 Contínuas e 1 Discreta.
* Temos 6 variáveis Qualitativas, divididas igualmente em Nominal e Ordinal.


#### 1.3 Observando o conjunto amostral

In [31]:
# Lista valores por variável
uniques = {'Variavel': [], 'Tipo': [], 'Valores': []}

for variable in dict['variavel']:
    if variable in df.columns:
        uniques['Variavel'].append(variable)
        uniques['Tipo'].append(df[variable].dtype)
        uniques['Valores'].append(set(df[variable].unique()))
    else:
        uniques['Variavel'].append(variable)
        uniques['Tipo'].append(None)
        uniques['Valores'].append(set())

df_uniques = pd.DataFrame.from_dict(uniques)
display(HTML(df_uniques.to_html())) # exibe

,Variavel,Tipo,Valores
0,Size,None,{}
1,Weight,None,{}
2,Brix,None,{}
3,pH,None,{}
4,Softness,None,{}
5,HarvestTime,None,{}
6,Ripeness,None,{}
7,Color,object,"{Orange-Red, Deep Orange, Yellow-Orange, Light Orange, Orange}"
8,Variety,object,"{Valencia, Navel (Early Season), Jaffa, Hamlin, Tangelo (Hybrid), Washington Navel, Moro (Blood), Star Ruby, Clementine, California Valencia, Midsweet (Hybrid), Cara Cara, Clementine (Seedless), Satsuma Mandarin, Temple, Navel, Navel (Late Season), Murcott (Hybrid), Blood Orange, Ambiance, Minneola (Hybrid), Ortanique (Hybrid), Tangerine, Honey Tangerine}"
9,Blemishes,None,{}


#### 1.4 Analisando valores faltantes

In [30]:
df.info()

qtd_nulos = df.isnull().sum()
qtd_nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Size (cm)           241 non-null    float64
 1   Weight (g)          241 non-null    int64  
 2   Brix (Sweetness)    241 non-null    float64
 3   pH (Acidity)        241 non-null    float64
 4   Softness (1-5)      241 non-null    float64
 5   HarvestTime (days)  241 non-null    int64  
 6   Ripeness (1-5)      241 non-null    float64
 7   Color               241 non-null    object 
 8   Variety             241 non-null    object 
 9   Blemishes (Y/N)     241 non-null    object 
 10  Quality (1-5)       241 non-null    float64
dtypes: float64(6), int64(2), object(3)
memory usage: 20.8+ KB


Size (cm)             0
Weight (g)            0
Brix (Sweetness)      0
pH (Acidity)          0
Softness (1-5)        0
HarvestTime (days)    0
Ripeness (1-5)        0
Color                 0
Variety               0
Blemishes (Y/N)       0
Quality (1-5)         0
dtype: int64

- Observamos, então, que o conjunto de dados está completo e não apresenta valores ausentes.

#### 1.5 Outliers

#### 1.6 Normalização dos Dados

---

### 2. Preparação de Dados

aaaaaa

#### 2.1 Normalização dos Dados

#### 2.1 Tratamento de Dados

In [ ]:
# tratamento de dados discrepantes
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

model = LogisticRegression()

## Configuração do experimento

## Resultados 

## Discussão